### Neural Computation Paper Figures

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FormatStrFormatter
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap

In [ ]:
fontsize = 18
figsize = (10,10)

In [ ]:
class lca_512_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_512_vh"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_768_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_vh"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_1024_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_1024_vh"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False

class sae_768_vh_params(object):
  def __init__(self):
    self.model_type = "sae"
    self.model_name = "sae_768_vh"
    self.version = "0.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class rica_768_vh_params(object):
  def __init__(self):
    self.model_type = "rica"
    self.model_name = "rica_768_vh"
    self.version = "0.0"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class lca_768_mnist_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class sae_768_mnist_params(object):
  def __init__(self):
    self.model_type = "sae"
    self.model_name = "sae_768_mnist"
    self.version = "1.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

In [ ]:
params_list = [lca_768_vh_params()]#, rica_768_vh_params(), sae_768_vh_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)
  analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.model.setup(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

### Iso-Response Contours

In [ ]:
min_angle = 20
max_angle = 55

num_neurons = 2 # How many neurons to plot

for analyzer in analyzer_list:
  analyzer.neuron_angles, analyzer.plot_matrix = analyzer.get_neuron_angles(analyzer.bf_stats)

  vectors = np.argwhere(np.logical_and(analyzer.plot_matrix<max_angle, analyzer.plot_matrix>min_angle))
  
  analyzer.target_neuron_ids = []
  analyzer.comparison_neuron_ids = []
  analyzer.target_vectors = []
  analyzer.comparison_vectors = []
  analyzer.rand_orth_vectors = []
  analyzer.comp_orth_vectors = []
  for vector_id in range(num_neurons):
    analyzer.target_neuron_ids.append(vectors[vector_id, 0])

    target_vector = analyzer.bf_stats["basis_functions"][analyzer.target_neuron_id]
    target_vector = target_vector.reshape(analyzer.model_params.num_pixels)
    target_vector_norm = np.linalg.norm(target_vector)
    analyzer.target_vectors.append(target_vector / target_vector_norm)

    analyzer.rand_orth_vectors.append(dp.get_rand_vectors(analyzer.target_vector, analyzer.model.num_pixels-1))
    
    sub_comparison_vectors = []
    sub_comparison_orth_vectors = []
    sub_comparison_neuron_ids = [index for index in range(analyzer.bf_stats["num_outputs"]) if index != vector_id]
    for comparison_neuron_id in sub_comparison_neuron_ids:
      if(analyzer.analysis_params.model_type.lower() == "lca"): # TODO: any 1 layer model would be fine - check num layers or something
        comparison_vector = analyzer.bf_stats["basis_functions"][comparison_neuron_id]
      else:
        comparison_vector = analyzer.optimal_stims[comparison_neuron_id]
      comparison_vector = comparison_vector.reshape(analyzer.model_params.num_pixels)
      comparison_vector = comparison_vector / np.linalg.norm(comparison_vector)
      sub_comparison_vectors.append(comparison_vector)
      sub_comparison_orth_vectors.append(dp.get_alt_vectors(target_vector, comparison_vector)[0])
    analyzer.comparison_neuron_ids.append(sub_comparison_neuron_ids)
    analyzer.comparison_vectors.append(sub_comparison_vectors)
    analyzer.comparison_orth_vectors.append(sub_comparison_orth_vectors)

In [ ]:
def get_norm_activity(analyzer, x_range, y_range, neuron_id_list, target_vect_list, comparison_vect_list, num_imgs):
  # Construct point dataset
  x_pts = np.linspace(x_range[0], x_range[1], int(np.sqrt(num_imgs)))
  y_pts = np.linspace(y_range[0], y_range[1], int(np.sqrt(num_imgs)))
  
  X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
  proj_datapoints = np.stack([X_mesh.reshape(num_imgs), Y_mesh.reshape(num_imgs)], axis=1)

  out_dict = {
    "norm_activity": [],
    "proj_target_neuron": [],
    "proj_comparison_neuron": [],
    "proj_orth_vect": [],
    "orth_vect": [],
    "proj_datapoints": proj_datapoints,
    "X_mesh": X_mesh,
    "Y_mesh": Y_mesh}

  # TODO: This can be made to be much faster by compiling all of the stimulus into a single set and computing activations
  for neuron_id, target_vect in zip(neuron_id_list, target_vect_list):
    activity_sub_list = []
    proj_target_neuron_sub_list = []
    proj_comparison_neuron_sub_list = []
    proj_orth_vect_sub_list = []
    orth_vect_sub_list = []
    for comparison_vect in comparison_vect_list:
      proj_matrix, orth_vect = dp.bf_projections(target_vect, comparison_vect)
      proj_target_neuron_sub_list.append(np.dot(proj_matrix, target_vect).T) #project
      proj_comparison_neuron_sub_list.append(np.dot(proj_matrix, comparison_vect).T) #project
      proj_orth_vect_sub_list.append(np.dot(proj_matrix, orth_vect).T) #project
      orth_vect_sub_list.append(orth_vect)
      datapoints = np.stack([np.dot(proj_matrix.T, proj_datapoints[data_id,:])
        for data_id in range(num_imgs)], axis=0) #inject
      datapoints = dp.reshape_data(datapoints, flatten=False)[0]
      datapoints = {"test": Dataset(datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
      datapoints = analyzer.model.reshape_dataset(datapoints, analyzer.model_params)
      activations = analyzer.compute_activations(datapoints["test"].images)#, batch_size=int(np.sqrt(num_imgs)))
      activations = activations[:, neuron_id]
      activity_max = np.amax(np.abs(activations))
      activations = activations / (activity_max + 0.00001)
      activations = activations.reshape(int(np.sqrt(num_imgs)), int(np.sqrt(num_imgs)))
      activity_sub_list.append(activations)
    out_dict["norm_activity"].append(activity_sub_list)
    out_dict["proj_target_neuron"].append(proj_target_neuron_sub_list)
    out_dict["proj_comparison_neuron"].append(proj_comparison_neuron_sub_list)
    out_dict["proj_orth_vect"].append(proj_orth_vect_sub_list)
    out_dict["orth_vect"].append(orth_vect_sub_list)
  return out_dict

### Orientation Selectivity

In [ ]:
min_rads = []
max_rads = []
for analyzer in analyzer_list:
  analyzer.bf_spatial_freq_rads = [np.sqrt(x**2+y**2) for (y,x) in analyzer.bf_stats["fourier_centers"]]
  min_rads.append(np.min(analyzer.bf_spatial_freq_rads))
  max_rads.append(np.max(analyzer.bf_spatial_freq_rads))
  
num_bins = 10
min_rad = np.min(min_rads)
max_rad = np.max(max_rads)
bins = np.linspace(min_rad, max_rad, num_bins)
fig, ax = plt.subplots(1, figsize=figsize)
hist_max = []
for analyzer in analyzer_list:
  hist, bin_edges = np.histogram(analyzer.bf_spatial_freq_rads, bins, density=True)
  bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
  bin_centers = bin_left + (bin_right - bin_left)/2
  
  label = analyzer.model.params.model_type.upper() + " " + str(analyzer.model.get_num_latent())# + " van Hateren"
  #label = re.sub("_", " ", analyzer.model_name)
  ax.plot(bin_centers, hist, alpha=1.0, linestyle="-", drawstyle="steps-mid", label=label)
  hist_max.append(np.max(hist))
  
ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::4], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.0f"))
ax.tick_params("both", labelsize=16)
ax.set_xlim([min_rad, max_rad])
ax.set_xticks([0, int(np.floor(max_rad/4)), int(2*np.floor(max_rad/4)),
  int(3*np.floor(max_rad/4)), max_rad])
ax.set_ylim([0, 0.6])
ax.set_xlabel("Spatial Frequency", fontsize=fontsize)
ax.set_ylabel("Normalized Density", fontsize=fontsize)
ax.set_title("Neuron Weight Spatial Frequency Histogram", fontsize=fontsize)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, labels, fontsize=fontsize, ncol=3,
  borderaxespad=0., bbox_to_anchor=[0.01, 0.99], fancybox=True, loc="upper left")
for line in legend.get_lines():
  line.set_linewidth(3)
plt.show()    